<a href="https://colab.research.google.com/github/explorer0008/ECDH-AES-GCM-encryption-technique/blob/main/encryption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pycryptodome

In [ ]:
from google.colab import files
import os

print("Please upload the file named **BATA** (or any file – it will be renamed).")
uploaded = files.upload()

# Take the first uploaded file and rename it to BATA
input_filename = list(uploaded.keys())[0]
os.rename(input_filename, "BATA")
print(f"Uploaded file saved as: BATA")

Please upload the file named **BATA** (or any file – it will be renamed).


Saving color.pdf to color.pdf
Uploaded file saved as: BATA


In [ ]:
from getpass import getpass

password = getpass("Enter encryption/decryption password: ")

Enter encryption/decryption password: ··········


In [ ]:
# === FIXED ECDH + AES-GCM (Colab-Compatible) ===
import base64
from pathlib import Path
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Protocol.KDF import PBKDF2
from Crypto.PublicKey import ECC
from Crypto.Hash import SHA256
from google.colab import files
import os

BATA = Path("BATA")
PRIVATE_KEY = Path("private.key")
PUBLIC_KEY = Path("public.key")

# FIXED: Use .d and .pointQ instead of ._key
def derive_shared_secret(private_key, peer_public_key):
    shared_point = private_key.d * peer_public_key.pointQ
    return shared_point.x.to_bytes(32, 'big')

def load_or_generate_keys():
    if PRIVATE_KEY.exists() and PUBLIC_KEY.exists():
        private_key = ECC.import_key(PRIVATE_KEY.read_bytes())
        public_key = ECC.import_key(PUBLIC_KEY.read_bytes())
        print("Loaded existing keys.")
    else:
        private_key = ECC.generate(curve='P-256')
        public_key = private_key.public_key()
        PRIVATE_KEY.write_bytes(private_key.export_key(format='DER'))
        PUBLIC_KEY.write_bytes(public_key.export_key(format='DER'))
        print("Generated new ECDH keypair.")
    return private_key, public_key

def derive_aes_key(shared_secret: bytes, password: str, salt: bytes) -> bytes:
    return PBKDF2(password.encode() + shared_secret, salt, 32, count=100000, hmac_hash_module=SHA256)

# ENCRYPT
def encrypt():
    if not BATA.exists():
        print("BATA not found! Upload it first.")
        uploaded = files.upload()
        os.rename(list(uploaded.keys())[0], "BATA")
    private_key, public_key = load_or_generate_keys()
    salt = get_random_bytes(16)
    nonce = get_random_bytes(12)
    shared_secret = derive_shared_secret(private_key, public_key)
    key = derive_aes_key(shared_secret, password, salt)
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)

    out_file = Path("BATA.enc")
    with open(out_file, 'wb') as f_out, open(BATA, 'rb') as f_in:
        f_out.write(salt + nonce + BATA.stat().st_size.to_bytes(8, 'big') + shared_secret)
        while chunk := f_in.read(64*1024):
            f_out.write(cipher.encrypt(chunk))
        f_out.write(cipher.digest())
    print("Encrypted! Downloading BATA.enc")
    files.download("BATA.enc")

# DECRYPT
def decrypt():
    enc_file = Path("BATA.enc")
    if not enc_file.exists():
        print("BATA.enc not found! Upload it:")
        uploaded = files.upload()
        os.rename(list(uploaded.keys())[0], "BATA.enc")
        enc_file = Path("BATA.enc")

    private_key, _ = load_or_generate_keys()
    with open(enc_file, 'rb') as f:
        salt = f.read(16)
        nonce = f.read(12)
        size = int.from_bytes(f.read(8), 'big')
        shared = f.read(32)
        ct = f.read(size)
        tag = f.read(16)

    key = derive_aes_key(shared, password, salt)
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    try:
        pt = cipher.decrypt_and_verify(ct, tag)
        Path("BATA.dec").write_bytes(pt)
        print("Decrypted! Downloading BATA.dec")
        files.download("BATA.dec")
    except ValueError:
        print("Failed: Wrong password or corrupted file")

# === RUN ===
action = input("Type 'encrypt' or 'decrypt': ").strip().lower()
if action == "encrypt":
    encrypt()
elif action == "decrypt":
    decrypt()
else:
    print("Invalid choice.")

Type 'encrypt' or 'decrypt': encrypt
Loaded existing keys.
Encrypted! Downloading BATA.enc


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>